#  Notebook 0.8: How Python Actually Runs on a Computer

###  Goal:
Understand the internal steps Python takes to run your code — from source to machine instructions.

###  Section 1: Python Execution Pipeline

Python follows this path to execute your code:

```text
Python Source Code (.py)
   ↓
Tokenizer → Parser → Abstract Syntax Tree (AST)
   ↓
Compiler → Bytecode (.pyc)
   ↓
Python Virtual Machine (PVM)
   ↓
Machine-level instructions (via C backend)
```

- Each stage transforms and prepares the code for efficient execution.

###  Section 2: Tokenization and Parsing

In [1]:
import tokenize
from io import BytesIO

code = b"x = 3 + 4"
for tok in tokenize.tokenize(BytesIO(code).readline):
    print(tok)

TokenInfo(type=63 (ENCODING), string='utf-8', start=(0, 0), end=(0, 0), line='')
TokenInfo(type=1 (NAME), string='x', start=(1, 0), end=(1, 1), line='x = 3 + 4')
TokenInfo(type=54 (OP), string='=', start=(1, 2), end=(1, 3), line='x = 3 + 4')
TokenInfo(type=2 (NUMBER), string='3', start=(1, 4), end=(1, 5), line='x = 3 + 4')
TokenInfo(type=54 (OP), string='+', start=(1, 6), end=(1, 7), line='x = 3 + 4')
TokenInfo(type=2 (NUMBER), string='4', start=(1, 8), end=(1, 9), line='x = 3 + 4')
TokenInfo(type=4 (NEWLINE), string='', start=(1, 9), end=(1, 10), line='')
TokenInfo(type=0 (ENDMARKER), string='', start=(2, 0), end=(2, 0), line='')


- Tokenization splits code into **identifiers, numbers, operators**.
- Parser builds an **AST** (abstract syntax tree) from those tokens.

In [2]:
import ast

expr = "x = 3 + 4"
tree = ast.parse(expr)
ast.dump(tree, indent=4)

"Module(\n    body=[\n        Assign(\n            targets=[\n                Name(id='x', ctx=Store())],\n            value=BinOp(\n                left=Constant(value=3),\n                op=Add(),\n                right=Constant(value=4)))],\n    type_ignores=[])"

###  Section 3: Compilation to Bytecode

In [3]:
import dis

def f():
    x = 3 + 4
    return x

dis.dis(f)

  4           0 LOAD_CONST               1 (7)
              2 STORE_FAST               0 (x)

  5           4 LOAD_FAST                0 (x)
              6 RETURN_VALUE


- Python is **compiled to bytecode**, not machine code.
- This bytecode is platform-independent and executed by the **Python Virtual Machine**.

###  Section 4: Python Virtual Machine (PVM)

- The PVM is a **stack-based interpreter** written in C.
- It reads `.pyc` files and runs bytecode instructions like `LOAD_CONST`, `BINARY_ADD`, etc.
- Located in `ceval.c` in CPython source.

```text
Bytecode → PVM loop (C) → Machine Instructions
```

- Each bytecode op is handled by a big switch-case loop in C:
```c
switch(opcode) {
  case LOAD_CONST:
    ...
  case BINARY_ADD:
    ...
}
```

###  Section 5: .pyc Files and Caching

- When a `.py` file runs, Python generates a `.pyc` file in `__pycache__/`
- These contain the compiled bytecode
- They speed up startup since parsing/compiling isn't needed again

```bash
__pycache__/example.cpython-310.pyc
```

###  Summary

| Stage        | Description                                     |
|--------------|-------------------------------------------------|
| Tokenization | Convert text into symbols (x, +, =)             |
| Parsing      | Build a syntax tree (AST)                      |
| Compilation  | Turn AST into bytecode                         |
| PVM          | Run bytecode using stack-based interpreter     |
| Machine Code | Generated indirectly through C backend         |

**Python is not just interpreted — it compiles to bytecode and runs on a virtual machine.**